## Load libraries

In [ ]:
import json
import urllib
import requests
import polling

## Set token/URL

The Python `requests` library expects a single URL, rather than a URL plus a path, hence we define a URL that includes the path to the simulation calls.

In [ ]:
token_file = "token.json"
base_path = "https://covid-modelling-stg.epcc.ed.ac.uk/api"
simulations_url = "/".join([base_path, "simulations"])

In [ ]:
with open(token_file,'r') as f:
    token = json.load(f)["token"]

In [ ]:
headers = {"accept":"application/json; charset=utf-8", "Content-Type": "application/json", "Authorization": "Bearer "+ token}

## Input

In [ ]:
inputJSON = {
  "model_slug": "sir-ode-python",
  "config": {
    "p": [
      0.5,
      0.25
    ],
    "u0": [
      0.99,
      0.01,
      0
    ],
    "tspan": [
      0,
      10000
    ]
  }
}

These are parameters to poll for output (every `step` seconds) and to timeout the simulation.

In [ ]:
step = 60
timeout = 600

## Post simulations

In [ ]:
post_simulations_response = requests.post(simulations_url, json = inputJSON, headers = headers)

In [ ]:
post_simulations_response.status_code

In [ ]:
post_simulations_content = post_simulations_response.json()
post_simulations_content

## Check simulations

In [ ]:
check_simulations_url = "/".join([simulations_url,str(post_simulations_response.json()["id"])])
check_simulations_url

In [ ]:
check_simulations_response = requests.get(check_simulations_url, headers = headers)

In [ ]:
check_simulations_response.status_code

In [ ]:
check_simulations_content = check_simulations_response.json()
check_simulations_content

## Export simulation output

In [ ]:
export_simulations_params = {
    "model": inputJSON["model_slug"],
    "format": "results"
}

In [ ]:
export_simulations_url = "/".join([simulations_url,str(post_simulations_response.json()["id"]),"export"])
export_simulations_url

The following code polls the API to check for success every `step` seconds, with a `timeout` in seconds.

In [ ]:
def is_200_response(response):
    """Check that the response returned 'success'"""
    return response.status_code == 200

try:
    export_simulations_response = polling.poll(
        lambda: requests.get(export_simulations_url, params = export_simulations_params, headers = headers),
        check_success=is_200_response,
        step=step,
        timeout=timeout)
except polling.TimeoutException:
    print(export_simulations_url+" failed")

In [ ]:
export_simulations_content = export_simulations_response.json()
export_simulations_content